# T1059.007 - JavaScript
Adversaries may abuse various implementations of JavaScript for execution. JavaScript (JS) is a platform-independent scripting language (compiled just-in-time at runtime) commonly associated with scripts in webpages, though JS can be executed in runtime environments outside the browser.(Citation: NodeJS)

JScript is the Microsoft implementation of the same scripting standard. JScript is interpreted via the Windows Script engine and thus integrated with many components of Windows such as the [Component Object Model](https://attack.mitre.org/techniques/T1559/001) and Internet Explorer HTML Application (HTA) pages.(Citation: JScrip May 2018)(Citation: Microsoft JScript 2007)(Citation: Microsoft Windows Scripts)

JavaScript for Automation (JXA) is a macOS scripting language based on JavaScript, included as part of Apple’s Open Scripting Architecture (OSA), that was introduced in OSX 10.10. Apple’s OSA provides scripting capabilities to control applications, interface with the operating system, and bridge access into the rest of Apple’s internal APIs. As of OSX 10.10, OSA only supports two languages, JXA and [AppleScript](https://attack.mitre.org/techniques/T1059/002). Scripts can be executed via the command line utility <code>osascript</code>, they can be compiled into applications or script files via <code>osacompile</code>, and they can be compiled and executed in memory of other programs by leveraging the OSAKit Framework.(Citation: Apple About Mac Scripting 2016)(Citation: SpecterOps JXA 2020)(Citation: SentinelOne macOS Red Team)(Citation: Red Canary Silver Sparrow Feb2021)(Citation: MDSec macOS JXA and VSCode)

Adversaries may abuse various implementations of JavaScript to execute various behaviors. Common uses include hosting malicious scripts on websites as part of a [Drive-by Compromise](https://attack.mitre.org/techniques/T1189) or downloading and executing these script files as secondary payloads. Since these payloads are text-based, it is also very common for adversaries to obfuscate their content as part of [Obfuscated Files or Information](https://attack.mitre.org/techniques/T1027).

## Atomic Tests

### Atomic Test #1 - JScript execution to gather local computer information via cscript
JScript execution test, execute JScript via cscript command. When successful, system information will be written to $env:TEMP\T1059.007.out.txt
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Sample script must exist on disk at specified location (#{jscript})
##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1059.007\src\sys_info.js") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -ItemType Directory (Split-Path "PathToAtomicsFolder\T1059.007\src\sys_info.js") -Force | Out-Null
Invoke-WebRequest "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1059.007/src/sys_info.js" -OutFile "PathToAtomicsFolder\T1059.007\src\sys_info.js"
```

In [ ]:
Invoke-AtomicTest T1059.007 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
cscript "PathToAtomicsFolder\T1059.007\src\sys_info.js" > %tmp%\T1059.007.out.txt
```

In [ ]:
Invoke-AtomicTest T1059.007 -TestNumbers 1

#### Cleanup: 
```cmd
del %tmp%\T1059.007.out.txt >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1059.007 -TestNumbers 1 -Cleanup

### Atomic Test #2 - JScript execution to gather local computer information via wscript
JScript execution test, execute JScript via wscript command. When successful, system information will be shown with four message boxes.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Sample script must exist on disk at specified location (#{jscript})
##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1059.007\src\sys_info.js") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -ItemType Directory (Split-Path "PathToAtomicsFolder\T1059.007\src\sys_info.js") -Force | Out-Null
Invoke-WebRequest "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1059.007/src/sys_info.js" -OutFile "PathToAtomicsFolder\T1059.007\src\sys_info.js"
```

In [ ]:
Invoke-AtomicTest T1059.007 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
wscript "PathToAtomicsFolder\T1059.007\src\sys_info.js"
```

In [ ]:
Invoke-AtomicTest T1059.007 -TestNumbers 2

## Detection
Monitor for events associated with scripting execution, such as process activity, usage of the Windows Script Host (typically cscript.exe or wscript.exe), file activity involving scripts, or loading of modules associated with scripting languages (ex: JScript.dll). Scripting execution is likely to perform actions with various effects on a system that may generate events, depending on the types of monitoring used. Monitor processes and command-line arguments for execution and subsequent behavior. Actions may be related to network and system information [Discovery](https://attack.mitre.org/tactics/TA0007), [Collection](https://attack.mitre.org/tactics/TA0009), or other programmable post-compromise behaviors and could be used as indicators of detection leading back to the source.

Monitor for execution of JXA through <code>osascript</code> and usage of <code>OSAScript</code> API that may be related to other suspicious behavior occurring on the system.

Understanding standard usage patterns is important to avoid a high number of false positives. If scripting is restricted for normal users, then any attempts to enable related components running on a system would be considered suspicious. If scripting is not commonly used on a system, but enabled, execution running out of cycle from patching or other administrator functions is suspicious. Scripts should be captured from the file system when possible to determine their actions and intent.